In [12]:
import pandas as pd
from src.utils import MySQLAgent, read_config

In [13]:
conn_path = ".env/connections.json"
configs = read_config(path=conn_path)
conn_configs = configs["CREDITREPORT"]['VM1_news_mysql_conn_info']
sql_agent = MySQLAgent(config=conn_configs)

In [14]:
company_name = "健策精密工業股份有限公司"
# AR
query = f"""
select company_name, trx_number, trx_date, due_date, date_applied, amount_due_original, amount_due_remaining
from erp_ar
where company_name = '{company_name}' AND due_date <= CURRENT_DATE()
"""

df_ar = sql_agent.read_table(query=query)

In [15]:
df_ar.shape

(1060, 7)

In [16]:
df_ar.trx_number.nunique()

1060

In [17]:
query = f"""
SELECT company_name, cash_receipt_id, receipt_number, trx_number, apply_date, amount_applied
FROM erp_app 
WHERE company_name = '{company_name}'
"""

df_app = sql_agent.read_table(query=query)

In [18]:
df_app.head()

,company_name,cash_receipt_id,receipt_number,trx_number,apply_date,amount_applied
0,健策精密工業股份有限公司,8277736.0,FT22003L7YK2,3854587_221080709,2022-01-03,23751.0
1,健策精密工業股份有限公司,8277736.0,FT22003L7YK2,3859442_221080951,2022-01-03,11682.0
2,健策精密工業股份有限公司,8277736.0,FT22003L7YK2,3864150_221080822,2022-01-03,1590.0
3,健策精密工業股份有限公司,8277736.0,FT22003L7YK2,3864151_221080945,2022-01-03,7649.0
4,健策精密工業股份有限公司,8277736.0,FT22003L7YK2,3864151_221080977,2022-01-03,18270.0


In [19]:
df_app.shape

(1166, 6)

In [20]:
query = f"""
SELECT company_name, receipt_number, receipt_method, receipt_date, clear_date, amount, currency_code
FROM erp_receipt
WHERE company_name = '{company_name}'
"""

df_receipt = sql_agent.read_table(query=query)

In [21]:
df_receipt.head()

,company_name,receipt_number,receipt_method,receipt_date,clear_date,amount,currency_code
0,健策精密工業股份有限公司,FT22003L7YK2,02-電匯,2022-01-03,2022-01-03,369250.0,TWD
1,健策精密工業股份有限公司,FT22038RMWJ8,02-電匯,2022-02-07,2022-02-07,431936.0,TWD
2,健策精密工業股份有限公司,FT22060GTTSW,02-電匯,2022-03-01,2022-03-01,416693.0,TWD
3,健策精密工業股份有限公司,FT2209067K9Z,02-電匯,2022-03-31,2022-03-31,422199.0,TWD
4,健策精密工業股份有限公司,FT22123M9RZ6,02-電匯,2022-05-03,2022-05-03,616933.0,TWD


In [22]:
df_ar_app = pd.merge(df_ar, df_app, how='left', on=['company_name', 'trx_number'])
df_ar_app_receipt = pd.merge(df_ar_app, df_receipt, how='left', on=['company_name', 'receipt_number'])
df_ar_app_receipt.head()

,company_name,trx_number,trx_date,due_date,date_applied,amount_due_original,amount_due_remaining,cash_receipt_id,receipt_number,apply_date,amount_applied,receipt_method,receipt_date,clear_date,amount,currency_code
0,健策精密工業股份有限公司,4260416_221121559,2022/01/04,2022/04/30,2022/05/03,18270.0,0.0,8358218.0,FT22123M9RZ6,2022-05-03,18270.0,02-電匯,2022-05-03,2022-05-03,616933.0,TWD
1,健策精密工業股份有限公司,4260416_221122341,2022/01/04,2022/04/30,2022/05/03,9664.0,0.0,8358218.0,FT22123M9RZ6,2022-05-03,9664.0,02-電匯,2022-05-03,2022-05-03,616933.0,TWD
2,健策精密工業股份有限公司,4260416_221122343,2022/01/04,2022/04/30,2022/05/03,17640.0,0.0,8358218.0,FT22123M9RZ6,2022-05-03,17640.0,02-電匯,2022-05-03,2022-05-03,616933.0,TWD
3,健策精密工業股份有限公司,4279913_221122392,2022/01/10,2022/04/30,2022/05/03,31295.0,0.0,8358218.0,FT22123M9RZ6,2022-05-03,31295.0,02-電匯,2022-05-03,2022-05-03,616933.0,TWD
4,健策精密工業股份有限公司,4279913_221122394,2022/01/10,2022/04/30,2022/05/03,18270.0,0.0,8358218.0,FT22123M9RZ6,2022-05-03,18270.0,02-電匯,2022-05-03,2022-05-03,616933.0,TWD
